In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import psycopg2.extras as extras
from ast import literal_eval
from psycopg2.extensions import register_adapter, AsIs

psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

pd.options.display.max_columns = 100

## Connect to Database

In [ ]:
# Enter credentials
host = ####
database = ####
user = ####
password = ####

def create_connection(host, database, user, password):
    return psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password)

connection = create_connection(host, database, user, password)

## Clear Database

In [ ]:
cursor = connection.cursor()

drop_table_query = '''
    DROP TABLE IF EXISTS recipeIngredientJoin;
    DROP TABLE IF EXISTS recipeCleanJoin;
    DROP TABLE IF EXISTS rawIngredients;
    DROP TABLE IF EXISTS ingredients;
    DROP TABLE IF EXISTS cleanIngredients;
    DROP TABLE IF EXISTS recipeTagJoin;
    DROP TABLE IF EXISTS tags;
    DROP TABLE IF EXISTS interactions;
    DROP TABLE IF EXISTS users;
    DROP TABLE IF EXISTS recipes;
'''
    
cursor.execute(drop_table_query)
connection.commit()
print("Tables successfully deleted.")

## Insert Data into Tables

In [ ]:
def insert_data(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    Code from: https://naysan.ca/2020/05/09/pandas-to-postgresql-using-psycopg2-bulk-insert-performance-benchmark/
    """
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    
    print("Inserted data into {} table.".format(table))
    cursor.close()

### Recipes

In [ ]:
# Create table
cursor = connection.cursor()

create_table_query = '''
    DROP TABLE IF EXISTS recipes;
    
    CREATE TABLE IF NOT EXISTS recipes (
      recipeID INTEGER PRIMARY KEY,
      name VARCHAR(255),
      minutes INTEGER,
      contributorID INTEGER,
      submittedDate DATE,
      tagsList TEXT [],
      nutritionList FLOAT [],
      numSteps INTEGER,
      stepsList TEXT [],
      description TEXT,
      cleanIngredientNameList TEXT [],
      numIngredients INTEGER,
      i FLOAT,
      nameTokenList INTEGER [],
      ingredientTokenList TEXT,
      stepTokenList INTEGER [],
      techniqueList INTEGER [],
      calorieLevel FLOAT,
      cleanIngredientIDList INTEGER [],
      isComplete BOOLEAN,
      calories FLOAT,
      fatPDV FLOAT,
      sugarPDV FLOAT,
      sodiumPDV FLOAT,
      proteinPDV FLOAT,
      saturatedFatPDV FLOAT,
      carbsPDV FLOAT
    );
'''

cursor.execute(create_table_query)
connection.commit()
print("Table created successfully in PostgreSQL ")

In [ ]:
# Insert data
recipes = pd.read_csv("Clean Recipe Data/recipes.csv", parse_dates=["submittedDate"])
# nonblank_recipes = recipes.copy().dropna(how="any", axis=0)

# Convert list columns to lists
list_columns = ["tagsList", "nutritionList", "stepsList", "cleanIngredientNameList", \
                "nameTokenList", "stepTokenList", "techniqueList", "cleanIngredientIDList"]
print("Converting columns to list:")
for col in list_columns:
    print(col)
    # Replace nan with an empty list
    recipes.loc[pd.isnull(recipes[col]), col] = "[]"
    recipes[col] = recipes[col].apply(lambda x: literal_eval(str(x)))
    
insert_data(connection, recipes, "recipes")

### Clean Ingredients

In [ ]:
# Create table
cursor = connection.cursor()

create_table_query = '''
    CREATE TABLE IF NOT EXISTS cleanIngredients (
      cleanIngredientID INTEGER PRIMARY KEY,
      name VARCHAR(255)
    );
'''

cursor.execute(create_table_query)
connection.commit()
print("Table created successfully in PostgreSQL ")

In [ ]:
# Insert data
cleanIngredients = pd.read_csv("Clean Recipe Data/cleanIngredients.csv")
# cleanIngredients.head()

insert_data(connection, cleanIngredients, "cleaningredients")

### Raw Ingredients

In [ ]:
# Create table
cursor = connection.cursor()

create_table_query = '''
    DROP TABLE IF EXISTS rawIngredients;
    
    CREATE TABLE IF NOT EXISTS rawIngredients (
      rawIngredientID INTEGER PRIMARY KEY,
      cleanIngredientID INTEGER,
      rawName VARCHAR(255),
      rawNameLength INTEGER,
      processedName VARCHAR(255),
      processedNameLength INTEGER,
      FOREIGN KEY (cleanIngredientID) REFERENCES cleanIngredients (cleanIngredientID)
    );
'''

cursor.execute(create_table_query)
connection.commit()
print("Table created successfully in PostgreSQL ")

In [ ]:
# Insert data
rawIngredients = pd.read_csv("Clean Recipe Data/rawIngredients.csv")
# rawIngredients.head()

insert_data(connection, rawIngredients, "rawingredients")

### Recipe Clean Join

In [ ]:
# Create table
cursor = connection.cursor()

create_table_query = '''
    DROP TABLE IF EXISTS recipeCleanJoin;

    CREATE TABLE IF NOT EXISTS recipeCleanJoin (
       recipeID INTEGER NOT NULL,
       cleanIngredientID INTEGER NOT NULL,
       PRIMARY KEY (recipeID, cleanIngredientID),
       FOREIGN KEY (recipeID) REFERENCES recipes (recipeID),
       FOREIGN KEY (cleanIngredientID) REFERENCES cleanIngredients (cleanIngredientID)
    );
'''

cursor.execute(create_table_query)
connection.commit()
print("Table created successfully in PostgreSQL ")  

In [ ]:
# Insert data
recipeCleanJoin = pd.read_csv("Clean Recipe Data/recipeCleanJoin.csv")
recipeCleanJoin.drop_duplicates(inplace=True)
# recipeCleanJoin.head()

insert_data(connection, recipeCleanJoin, "recipeCleanJoin")

### Tags

In [ ]:
# Create table
cursor = connection.cursor()

create_table_query = '''
    DROP TABLE IF EXISTS tags;
    
    CREATE TABLE IF NOT EXISTS tags (
      tagID INTEGER PRIMARY KEY,
      tag VARCHAR(255),
      numRecipes INTEGER
    );
'''

cursor.execute(create_table_query)
connection.commit()
print("Table created successfully in PostgreSQL ")

In [ ]:
# Insert data
tags = pd.read_csv("Clean Recipe Data/tags.csv")
# tags.head()

insert_data(connection, tags, "tags")

### Recipe Tag Join

In [ ]:
# Create table
cursor = connection.cursor()

create_table_query = '''
    DROP TABLE IF EXISTS recipeTagJoin;

    CREATE TABLE IF NOT EXISTS recipeTagJoin (
       recipeID INTEGER NOT NULL,
       tagID INTEGER NOT NULL,
       PRIMARY KEY (recipeID, tagID),
       FOREIGN KEY (recipeID) REFERENCES recipes (recipeID),
       FOREIGN KEY (tagID) REFERENCES tags (tagID)
    );
'''

cursor.execute(create_table_query)
connection.commit()
print("Table created successfully in PostgreSQL ") 

In [ ]:
# Insert data
recipeTagJoin = pd.read_csv("Clean Recipe Data/recipeTagJoin.csv")
recipeTagJoin.drop_duplicates(inplace=True)
# recipeTagJoin.head()

n = 50000

for g, df in recipeTagJoin.groupby(np.arange(len(recipeTagJoin)) // n):
    
    # Recreate connection to avoid time-out
    connection = create_connection(host, database, user, password)

    # Insert group of rows
    insert_data(connection, df, "recipeTagJoin")
    print("Inserted {} records.".format(n * g + df.shape[0]))
    
    # Close connection
    if(connection):
        connection.close()
#         print("PostgreSQL connection is closed")

### Users

In [ ]:
# Create table
connection = create_connection(host, database, user, password)

cursor = connection.cursor()

create_table_query = '''
    DROP TABLE IF EXISTS users;
    
    CREATE TABLE IF NOT EXISTS users (
      userID INTEGER PRIMARY KEY,
      techniquesList INTEGER [],
      itemsList INTEGER [],
      numItems INTEGER,
      ratingsList FLOAT [],
      numRatings INTEGER
    );
'''

cursor.execute(create_table_query)
connection.commit()
print("Table created successfully in PostgreSQL ")

In [ ]:
# Insert data
users = pd.read_csv("Clean Recipe Data/users.csv")

# Convert list columns to lists
list_columns_users = ["techniquesList", "itemsList", "ratingsList"]
print("Converting columns to list:")
for col in list_columns_users:
    print(col)
    # Replace nan with an empty list
    users.loc[pd.isnull(users[col]), col] = "[]"
    users[col] = users[col].apply(lambda x: literal_eval(str(x)))

insert_data(connection, users, "users")

### Interactions

In [ ]:
# Create table
cursor = connection.cursor()

create_table_query = '''
    DROP TABLE IF EXISTS interactions;

    CREATE TABLE IF NOT EXISTS interactions (
      interactionID INTEGER PRIMARY KEY,
      recipeID INTEGER,
      userID INTEGER,
      date DATE,
      rating INTEGER,
      review TEXT,
      FOREIGN KEY (recipeID) REFERENCES recipes (recipeID)
    );
'''

cursor.execute(create_table_query)
connection.commit()
print("Table created successfully in PostgreSQL ") 

In [ ]:
# Interactions
interactions = pd.read_csv("Clean Recipe Data/interactions.csv")
# interactions.head()

n = 50000

for g, df in interactions.groupby(np.arange(len(interactions)) // n):
    
    # Recreate connection to avoid time-out
    connection = create_connection(host, database, user, password)

    # Insert group of rows
    insert_data(connection, df, "interactions")
    print("Inserted {} records.".format(n * g + df.shape[0]))
    
    # Close connection
    if(connection):
        connection.close()
#         print("PostgreSQL connection is closed")

In [ ]:
#closing database connection.
if(connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

In [ ]:
## Insert new table

## Add column to recipes

## Add new join table

### Ingredients

In [ ]:
# Create table
cursor = connection.cursor()

create_table_query = '''
    DROP TABLE IF EXISTS ingredients;
    
    CREATE TABLE IF NOT EXISTS ingredients (
      ingredientID INTEGER PRIMARY KEY,
      name VARCHAR(255)
    );
'''

cursor.execute(create_table_query)
connection.commit()
print("Table created successfully in PostgreSQL ")

In [ ]:
# Insert data
ingredients = pd.read_csv("Clean Recipe Data/ingredients.csv")
ingredients.head()

insert_data(connection, ingredients, "ingredients")

### Recipe Ingredient Join

In [ ]:
# Create table
cursor = connection.cursor()

create_table_query = '''
    DROP TABLE IF EXISTS recipeIngredientJoin;

    CREATE TABLE IF NOT EXISTS recipeIngredientJoin (
       recipeID INTEGER NOT NULL,
       ingredientID INTEGER NOT NULL,
       PRIMARY KEY (recipeID, ingredientID),
       FOREIGN KEY (recipeID) REFERENCES recipes (recipeID),
       FOREIGN KEY (ingredientID) REFERENCES ingredients (ingredientID)
    );
'''

cursor.execute(create_table_query)
connection.commit()
print("Table created successfully in PostgreSQL ")  

In [ ]:
# Insert data
recipeIngredientJoin = pd.read_csv("Clean Recipe Data/recipeIngredientJoin.csv")
recipeIngredientJoin.head()

insert_data(connection, recipeIngredientJoin, "recipeIngredientJoin")

## Add Column(s) to Recipe Table

In [ ]:
# Create the column
cursor = connection.cursor()

create_column_query = '''
    ALTER TABLE recipes
    ADD COLUMN ingredientIDList INTEGER [];
'''

cursor.execute(create_column_query)
connection.commit()
print("Column created successfully in PostgreSQL ")  

In [ ]:
# Format the data
recipeIngredientList = pd.read_csv("Clean Recipe Data/recipeIngredientList.csv")

# Convert list columns to lists
recipeIngredientList['ingredientIDList'] = recipeIngredientList['ingredientIDList'].apply(lambda x: literal_eval(str(x)))

recipeIngredientList.head()

In [ ]:
# Insert the data
# https://stackoverflow.com/questions/55052395/how-to-update-a-postgres-table-column-using-a-pandas-data-frame
cursor = connection.cursor()
rows = zip(recipeIngredientList.recipeID, recipeIngredientList.ingredientIDList)
cursor.execute("""CREATE TEMP TABLE ingredientList(id INTEGER, z INTEGER []) ON COMMIT DROP""")
cursor.executemany("""INSERT INTO ingredientList (id, z) VALUES(%s, %s)""", rows)

cursor.execute("""
    UPDATE recipes
    SET ingredientIDList = ingredientList.z
    FROM ingredientList
    WHERE ingredientList.id = recipes.recipeID;
    """)

cursor.rowcount
connection.commit()